In [2]:
import time
start = time.time()

In [3]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
from torch import optim
from torchvision import datasets, transforms, models
import json
import matplotlib.pyplot as plt
import torchvision
from torch import nn
from PIL import Image
import numpy as np
import os, random
from collections import OrderedDict
import matplotlib.image as mpimg
from torch.utils.data.dataset import Dataset  
from torchvision import transforms  
import pandas as pd  
import csv
import torch.optim.lr_scheduler


def find_classes(dataframe):
    classes = list(set(dataframe.iloc[:,1].values))
    classes.sort() 
    class_to_idx={classes[i]: i for i in range(len(classes))}
    return classes, class_to_idx    
    
class AnimalDataset(Dataset):
    def __init__(self, csv_file, data_dir, transform=None):
        self.csv_mapping = pd.read_csv(csv_file)
        
        self.root_dir = data_dir
        self.transform = transform
        self.classes, self.class_to_idx = find_classes(self.csv_mapping)
        
    def __len__(self):
        return len(self.csv_mapping)        
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.csv_mapping.iloc[idx, 0])
        image = Image.open(img_name) 
        image = image.convert("RGB")
        label = self.class_to_idx[self.csv_mapping.iloc[idx, 1]]
        
        if self.transform:
            image = self.transform(image)

        return (image, label)        
        
    def output_layer(self):
        return len(self.classes)
            
    
    def get_class_to_idx(self):
        return self.class_to_idx

In [4]:
train_transforms = transforms.Compose([transforms.RandomRotation(20),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])


train_data= AnimalDataset('F:/Project/HackerEarth/Identify the Animal/meta-data/train.csv',
                          'F:/Project/HackerEarth/Identify the Animal/train', transform = train_transforms)
                                           
                          
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64,shuffle=True)

In [5]:
model=models.vgg19(pretrained=True)
input_size = model.classifier[0].in_features

for param in model.parameters():
    param.requires_grad=False

    
outp= train_data.output_layer()
from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(input_size,4096)), #224*224=50176
                          ('relu', nn.ReLU()),
                         # ('dropout',nn.Dropout(0.2)),
                          ('fc2',nn.Linear(4096,2046)),
                          ('relu2', nn.ReLU()),
                         # ('dropout2',nn.Dropout(0.2)),
                          ('fc3',nn.Linear(2046,1024)),  
                          ('relu3', nn.ReLU()), 
                         # ('dropout',nn.Dropout(0.2)),
                          ('fc4',nn.Linear(1024,400)),  
                          ('relu4', nn.ReLU()), 
                        #  ('dropout4',nn.Dropout(0.2)),
                          ('fc5', nn.Linear(400,outp)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))

model.classifier=classifier
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.00005)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)


epochs=32
print_every =40
steps=0

exp_lr_scheduler=optim.lr_scheduler.MultiStepLR(optimizer, milestones=[13,22,29], gamma=0.1)
for e in range(epochs):
    model.train()  
    running_loss=0
    exp_lr_scheduler.step()
    for images, labels in iter(trainloader):
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        images,labels = images.to(device),labels.to(device)
    
        steps+=1
        images.resize_(images.shape[0],3,224,224)
        optimizer.zero_grad()
        output=model.forward(images)
        loss=criterion(output,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
        
        if steps%print_every==0:
            model.eval()
            
            print("Epoch: {}/{}.. ".format(e+1, epochs),
                  "Training Loss: {:.3f}.. ".format(running_loss/print_every))
            
            running_loss = 0
            model.train()
            
            
print(model)

Epoch: 1/32..  Training Loss: 2.619.. 
Epoch: 1/32..  Training Loss: 1.126.. 
Epoch: 1/32..  Training Loss: 0.846.. 
Epoch: 1/32..  Training Loss: 0.781.. 
Epoch: 1/32..  Training Loss: 0.669.. 
Epoch: 2/32..  Training Loss: 0.554.. 
Epoch: 2/32..  Training Loss: 0.643.. 
Epoch: 2/32..  Training Loss: 0.577.. 
Epoch: 2/32..  Training Loss: 0.570.. 
Epoch: 2/32..  Training Loss: 0.602.. 
Epoch: 3/32..  Training Loss: 0.473.. 
Epoch: 3/32..  Training Loss: 0.539.. 
Epoch: 3/32..  Training Loss: 0.533.. 
Epoch: 3/32..  Training Loss: 0.541.. 
Epoch: 3/32..  Training Loss: 0.549.. 
Epoch: 4/32..  Training Loss: 0.342.. 
Epoch: 4/32..  Training Loss: 0.511.. 
Epoch: 4/32..  Training Loss: 0.500.. 
Epoch: 4/32..  Training Loss: 0.479.. 
Epoch: 4/32..  Training Loss: 0.463.. 
Epoch: 5/32..  Training Loss: 0.260.. 
Epoch: 5/32..  Training Loss: 0.478.. 
Epoch: 5/32..  Training Loss: 0.429.. 
Epoch: 5/32..  Training Loss: 0.476.. 
Epoch: 5/32..  Training Loss: 0.465.. 
Epoch: 6/32..  Training L

In [6]:
model.class_to_idx = train_data.class_to_idx

checkpoint = {'in_features': 25088,
              'out_features': outp,
              'epoch': 3,
              'optimizer' : optimizer.state_dict(),
              'state_dict': model.state_dict(),
              'class_to_idx': train_data.class_to_idx,
              'classifier': classifier.state_dict(),
              'lr':0.001}

torch.save(checkpoint, 'F:/Project/HackerEarth/Identify the Animal/checkpoint_2.7.pth')       

In [7]:
def process_image(image):

    pil_image= Image.open(image)

    size=256,256
    pil_image.thumbnail(size)
    
    
    width,height = 256,256
    left = (width - 224)/2
    top = (height - 224)/2
    right = (width + 224)/2   
    bottom = (height + 224)/2    
    np_image = np.array(pil_image.crop((left, top, right, bottom))) 


    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])

    np_image=(np_image/255-mean)/std  #/255 is necessary
    np_image= np.transpose(np_image, (2, 0, 1))
    
    return np_image

def predict(image_path, model):

    
    np_array = process_image(image_path)
    tensor_image = torch.from_numpy(np_array).double()
    
    
    var_inputs = Variable(tensor_image)
    
    var_inputs = var_inputs.unsqueeze(0)
    var_inputs=var_inputs.float().to(device)

    output = model.forward(var_inputs)  
    ps=torch.exp(output)
    tn_probs = ps.cpu()
    probs=tn_probs.detach().numpy()
    probs=probs.flatten().tolist()  

    return probs 


probs = predict('F:/Project/HackerEarth/Identify the Animal/test/Img-1.jpg',model)

In [8]:
class_names = train_data.classes

In [9]:
class_header=class_names.copy()
class_header.insert(0,'image_id')

import csv
test_imgs_name= list(pd.read_csv('F:/Project/HackerEarth/Identify the Animal/test.csv').iloc[:,0])
with open('F:/Project/HackerEarth/Identify the Animal/predictionv2.7.csv', 'w') as f:
    writer = csv.writer(f,lineterminator = '\n')
    writer.writerow(class_header)
    for test_img in test_imgs_name:
        probs = predict(('F:/Project/HackerEarth/Identify the Animal/test/'+ test_img),model)
        probs.insert(0,test_img)
        writer.writerow(probs)
                    

In [10]:
end = time.time()
print(end - start)

11835.853644132614


In [11]:
#Score
#0.86119